In [7]:
import tkinter as tk
from tkinter import filedialog, messagebox
import librosa
import joblib
import numpy as np

# Özellik çıkarım fonksiyonu
def extract_features(file_path):
    # Ses dosyasını yükle
    y, sr = librosa.load(file_path, sr=None)  # sr=None, orijinal örnekleme hızını korur
    
    # MFCC özelliklerini çıkar
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)  # 13 MFCC katsayısı
    
    # MFCC'nin ortalamasını al
    mfcc_mean = np.mean(mfcc, axis=1)
    
    return mfcc_mean

# Tkinter arayüzü
def load_and_predict():
    # Kullanıcıdan bir dosya seçmesi istenir
    file_path = filedialog.askopenfilename(filetypes=[("MP3 Files", "*.mp3")])
    if not file_path:
        return  # Dosya seçilmediyse çık
    
    try:
        # Modeli yükle
        model = joblib.load("random_forest_model.pkl")
        
        # Özellikleri çıkar
        features = extract_features(file_path).reshape(1, -1)  # Tek boyutlu array
        
        # Tahmin yap
        prediction = model.predict(features)[0]
        
        # Tahmini sınıfa dönüştür
        instrument_classes = ["Piyano", "Keman", "Kanun", "Gitar"]
        predicted_instrument = instrument_classes[int(prediction)]
        
        # Sonucu kullanıcıya göster
        messagebox.showinfo("Tahmin Sonucu", f"Bu dosya '{predicted_instrument}' enstrümanına ait.")
    except Exception as e:
        messagebox.showerror("Hata", f"Tahmin sırasında bir hata oluştu: {e}")

# Tkinter GUI
root = tk.Tk()
root.title("Enstrüman Tahmini")

label = tk.Label(root, text="Bir MP3 dosyası yükleyin ve çalan enstrümanı öğrenin.")
label.pack(pady=20)

button = tk.Button(root, text="MP3 Dosyası Yükle", command=load_and_predict)
button.pack(pady=10)

root.mainloop()
